In [11]:
# Divide textos em pedaços (chunks) controlando tamanho/overlap
from langchain_text_splitters import RecursiveCharacterTextSplitter
# Embeddings e LLM da OpenAI (usados para vetorização e geração)
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
# Vetorstore local Chroma para armazenar/consultar embeddings
from langchain_community.vectorstores import Chroma
# Constrói prompts de chat com variáveis ("{question}", "{context}")
from langchain_core.prompts import ChatPromptTemplate

# Operadores de composição de cadeias (Passagem direta, Paralelo)
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
# Parser que transforma a saída do LLM em string simples
from langchain_core.output_parsers import StrOutputParser

# Carrega PDF e divide em Documentos de página
from langchain_community.document_loaders import PyPDFLoader
# Retriever que mantém relação pai↔filho entre pedaços (parent/child)
from langchain.retrievers import ParentDocumentRetriever
# Loja de documentos em memória para armazenar os pais
from langchain.storage import InMemoryStore


In [3]:
import os
# Define a chave da OpenAI via variável de ambiente
# As libs `OpenAIEmbeddings` e `ChatOpenAI` leem daqui automaticamente
os.environ["OPENAI_API_KEY"] = "sk-..."

In [4]:
# Cria função de embeddings da OpenAI (vetores para busca semântica)
# Modelo: 'text-embedding-3-small' (rápido e barato, 1536 dimensões)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
# LLM para gerar respostas; limita a saída para 200 tokens
llm_model = ChatOpenAI(model_name="gpt-3.5-turbo", max_tokens=200)

In [6]:
# Nome do arquivo PDF que será processado
pdf = "yan.pdf"
# Loader que lê o PDF e transforma em Documentos; sem extrair imagens
carregar_pdf = PyPDFLoader(pdf, extract_images=False)
# Carrega o PDF e divide por páginas em uma lista de Documentos
pages = carregar_pdf.load_and_split()
# Mostra quantas páginas foram carregadas
len(pages)

5

In [7]:
# Splitter dos 'filhos' (pedaços pequenos usados na busca/vetorstore)
# chunk_size=200: cada pedaço terá ~200 caracteres
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200)

# Splitter dos 'pais' (blocos maiores preservam contexto original)
# chunk_size=4000: tamanho de cada bloco pai
# chunk_overlap=200: sobreposição entre pedaços consecutivos para não perder contexto
# length_function=len: mede tamanho via número de caracteres
# add_start_index=True: inclui índice inicial de cada pedaço nos metadados
parent_splitter = RecursiveCharacterTextSplitter(
    chunk_size=4000,
    chunk_overlap=200,
    length_function=len,
    add_start_index=True,
    )

In [8]:
# Armazena documentos-pai em memória (rápido, volátil)
store = InMemoryStore()
# Cria vetorstore local Chroma com embeddings OpenAI
# persist_directory="childVectorDB": caminho do banco local (reabre entre execuções)
vectorstory = Chroma(embedding_function=embeddings, persist_directory="childVectorDB")
# Obs.: aviso de deprecação indica migrar para pacote langchain-chroma futuramente


/var/folders/5l/x5tjl9qn4kq7zrm6vh3twr300000gn/T/ipykernel_81055/3160842109.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstory = Chroma(embedding_function=embeddings, persist_directory="childVectorDB")


In [ ]:
# Retriever que cria pedaços 'filhos' para busca, mas retorna blocos 'pais'
# vectorstore: onde os filhos (chunks) são indexados/vetorizados
# docstore: onde os documentos-pai completos são guardados (InMemoryStore)
# child_splitter: como quebrar o texto em pedaços menores (para indexação)
# parent_splitter: como manter/recuperar o contexto maior (pais)
parent_document_retriever = ParentDocumentRetriever(
    vectorstore=vectorstory,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

# Indexa as páginas do PDF no retriever (gera filhos e relaciona com pais)
# ids=None: deixa o retriever criar IDs automaticamente
parent_document_retriever.add_documents(pages, ids=None)
# Dica: em retrievers baseados em Chroma simples, você pode usar search_type="mmr"
# para equilibrar relevância e diversidade nos resultados (evita retornos redundantes).


In [ ]:
# Inspeciona/obtém dados da coleção Chroma (útil para debug)
parent_document_retriever.vectorstore.get()

In [12]:
# Prompt com instrução e espaços para preencher pergunta e contexto
# {question}: será a dúvida do usuário | {context}: textos recuperados
TEMPLATE = """
Voce e um especialista em curriculos, responda a pergunta com base apenas no contexto fornecido.
query: {question}

context: {context}
"""
# Cria o objeto de prompt a partir do template acima
rag_prompt = ChatPromptTemplate.from_template(TEMPLATE)

In [13]:
# Prepara entradas paralelas para a cadeia: 
# - "question": repassa direto a pergunta do usuário
# - "context": usa o retriever para buscar textos relevantes
setup_retrival = RunnableParallel(
    {"question": RunnablePassthrough(), "context": parent_document_retriever}
)
# Converte a saída do LLM para string simples (sem JSON/estruturas)
output_parser = StrOutputParser()

In [14]:
# Monta a cadeia RAG:
# 1) setup_retrival produz {question, context}
# 2) rag_prompt injeta os dois no template
# 3) llm_model gera a resposta
# 4) output_parser extrai texto da resposta
parent_chain_retrieval = setup_retrival | rag_prompt | llm_model | output_parser

In [ ]:
# Faz a pergunta para a cadeia RAG; o retriever monta o contexto
parent_chain_retrieval.invoke("Qual é a função do candidato?")